# Доверительные интервалы и объем выборки

### Задача 1

* В вашей организации установлена некая система, которая считает доходность с каждого пользователя. Вы провели А/В тест с целью увеличить доход в двух группах variant и control. В приложенном excel файле ты найдешь сырые данные по результатам эксперимента (user_id), тип выборки (variant_name) и доход, принесенный пользователем (revenue). Проанализируйте результаты. С какой долей уверенности можно сказать, что доход изменился, достаточно ли данных для этого? Какой объем выборки нам необходим, чтобы мы могли сказать с 95% уверенностью, что доход вырос.  

In [1]:
import pandas as pd
import numpy as np

In [25]:
df = pd.read_excel('AB_Test_Results.xlsx')

df.head(10)

C:\Users\User\miniconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [3]:
df.tail(10)

,USER_ID,VARIANT_NAME,REVENUE
9990,4883,variant,0.0
9991,8864,variant,0.0
9992,9303,variant,0.0
9993,2400,variant,0.0
9994,3129,control,0.0
9995,1981,control,0.0
9996,502,variant,0.0
9997,9214,variant,0.0
9998,7741,control,0.0
9999,9468,control,0.0


In [4]:
# Проверим, что типы данных открылись корректно
df.dtypes

USER_ID           int64
VARIANT_NAME     object
REVENUE         float64
dtype: object

In [5]:
# Проверим наличие пустых значений
df.isnull().values.any()

False

In [6]:
# Проверим наличие дублей в USER_ID
df.duplicated(subset=['USER_ID'])

0       False
1       False
2       False
3       False
4       False
        ...  
9995     True
9996    False
9997     True
9998    False
9999    False
Length: 10000, dtype: bool

In [7]:
df[df['USER_ID'] == 9214]

,USER_ID,VARIANT_NAME,REVENUE
2342,9214,variant,0.0
6505,9214,control,0.0
9997,9214,variant,0.0


In [8]:
df.value_counts(subset=['USER_ID'])

USER_ID
9101       6
668        6
5652       6
8359       6
4879       6
          ..
4682       1
4684       1
4687       1
4689       1
5525       1
Length: 6324, dtype: int64

### Подитог

Часть USER_ID повторяются и присутствуют в обеих выборках (variant / control). Не очень понятно как это трактовать в контексте A/B теста, является ли это ошибкой и если нет, то как для "смешанных" пользователей записывается Revenue. Влияет ли как-то попадание пользователя в обе группы на приносимый им Revenue. Если да, то как. 

В реальности это повод запросить уточнение и\или провести дополнительные разбирательства. В рамках этого задания просто будем считать эти данные испорченными и не будем на них обращать внимание дальше. 

In [10]:
len(df)

10000

In [11]:
df_dd = df.drop_duplicates(subset=['USER_ID'], keep=False)

In [12]:
len(df_dd)

3664

In [14]:
df_dd.groupby(['VARIANT_NAME']).count()

,USER_ID,REVENUE
VARIANT_NAME,,
control,1834,1834
variant,1830,1830


In [13]:
df_dd.groupby(['VARIANT_NAME']).mean()

,USER_ID,REVENUE
VARIANT_NAME,,
control,5049.845692,0.182694
variant,4966.748087,0.066563


In [26]:
variant_mean = 0.066563
control_mean = 0.182694

In [27]:
df_dd.groupby(['VARIANT_NAME']).sum()

,USER_ID,REVENUE
VARIANT_NAME,,
control,9261417,335.06
variant,9089149,121.81


In [28]:
df_dd_va = df_dd[df_dd['VARIANT_NAME'] == 'variant'] #1830
df_dd_co = df_dd[df_dd['VARIANT_NAME'] == 'control'] #1834

In [29]:
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

In [30]:
variant_mean_std = df_dd_va['REVENUE'].std(ddof=1)/np.sqrt(len(df_dd_va))

In [31]:
variant_mean_std

0.02063481959392742

In [19]:
control_mean_std = df_dd_co['REVENUE'].std(ddof=1)/np.sqrt(len(df_dd_co))

In [32]:
control_mean_std

0.10865109018898764

Обратите внимание, что среднеквадратическое отклонение в control намного шире, чем в variant

In [20]:
# 95% двусторонний доверительный интервал для среднего REVENUE по группе variant

_tconfint_generic(variant_mean, 
                  variant_mean_std, 
                  len(df_dd_va) - 1,
                  0.05, 'two-sided')

(0.026092715331499275, 0.10703328466850072)

In [21]:
# 95% двусторонний доверительный интервал для среднего REVENUE по группе control

_tconfint_generic(control_mean, 
                  control_mean_std, 
                  len(df_dd_co) - 1,
                  0.05, 'two-sided')

(-0.03039893136648794, 0.3957869313664879)

Обратите внимание, как отличается доверительный интервал в двух выборках. Обратите внимание на отрезок пересечения двух интервалов. 

### Подитог 

На данной выборке среднее REVENUE выше в группе control на 0.1161. 

При этом 95% доверительные интервалы для среднего REVENUE выборок variant и control пересекаются на отрезке от 0 до 0.1070. 

Таким образом нельзя говорить о достоверной разнице средних REVENUE в двух выборках. 

Попробуем понять достоверную разницу для других вариантов доверительных интервалов, а также понять сколько нужно образцов для появления достоверного результата.

In [22]:
# 65% двусторонний доверительный интервал для среднего REVENUE по группе variant

_tconfint_generic(variant_mean, 
                  variant_mean_std, 
                  len(df_dd_va) - 1,
                  0.35, 'two-sided')

(0.04727297887565986, 0.08585302112434014)

In [23]:
# 65% двусторонний доверительный интервал для среднего REVENUE по группе control

_tconfint_generic(control_mean, 
                  control_mean_std, 
                  len(df_dd_co) - 1,
                  0.35, 'two-sided')

(0.08112390175477749, 0.2842640982452225)

### Попробуем понять, какой нужен объем выборки, чтобы достоверно говорить о разности средних

Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией $$\sigma^2$$:

$$\bar{X}_n\pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}$$

При σ=1 какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью ±0.01?


In [24]:
from scipy import stats
np.ceil((stats.norm.ppf(1-0.05/2) / 0.01)**2)

38415.0

Нам необходимо порядка 38400 образцов в двух выборках, чтобы с 95% уверенностью говорить о разнице средних с точностью 0,01